In [17]:
import requests
import pandas as pd

# Аналитика по резюме выпускников

## Подготовка к работе

Вся ниже представленная информация базируется на возможностях предоставляемых [HeadHunter API](https://dev.hh.ru/).

До начала работы **важно понимать**, что Вам потребуется аккаунт пользоваться, которые идентифицируется HH как работодатель. Это может быть как корпоративный аккаунт от университета (в рамках HH есть центральный HR-аккаунт, который может регистрировать новых корпоративных пользователей), так и совсем сторонний работодатель, с точки зрения забора данных это не важно. **Дополнительных финансовых вложений не потребуется**, мы будем использовать только данные резюме, которые требуют только авторизации как работодатель.

Дополнительно к аккаунту от работодателя Вам потребуется зарегистировать приложение для работы с API. Залогинтесь (зарегистрировать приложение можно под личным аккаунтом на HH, но рекомендую, для порядку, делать это всё из под одной корпоративной учётки) в [личном кабинете разработчика](https://dev.hh.ru/admin) и зарегистрируйте новое приложение. По моему опыту процесс одобрения приложения со стороны HH не превышает одной недели.

После регистрации приложения у него появятся следующие реквизиты: `Client ID`, `Client Secret`, `Токен приложения`. Это позволит Вам авторизовать пользователя и получить реквизиты для работы с API. Сам процесс авторизации неплохо описан самим HeadHunter в [статье на Habr-е](https://habr.com/ru/companies/hh/articles/303168/).

Надеюсь у Вас **всё получится** по выше описанному!

### ID университетов в HH

Для поиска ID-шников университетов, которые мы будем использовать для работы с API, вы можете использовать 3 способа:

1. [«Ручку» API HH](https://api.hh.ru/openapi/redoc#tag/Podskazki/operation/get-educational-institutions-suggests) для поиска по учебным заведениям. Вот [пример ссылки](https://api.hh.ru/suggests/educational_institutions?text=%D0%98%D0%A2%D0%9C%D0%9E) для поиска «ИТМО»;
2. [«Ручку» API psal.ru](https://app.psal.ru/api/university/), с данными об университетах, которые отслеживаются проектом (на момент написания, апрель 2023 года, это университеты входящие в 2 федеральных программы «Приоритет 2030» и «Передовые инженерные школы»);
3. Ещё можно воспользоваться [расширенным поиском](https://hh.ru/search/resume/advanced) на самом hh.ru, где есть UI-ки для указания учебного заведения. Вы можете воспользоваться DevTool браузера и при поиске/выборе учебного заведения, забирать необходимое Вам ID университета (см. gif-ку с иллюстрацией).

Я буду использовать вариант упомянутый в пункте 2 и для полноты картины остановлюсь на некоторых моментах:

1. У университетов есть филиалы, но в используемом API psa.ru используется только ID головных организаций. Пример с [МФТИ](https://api.hh.ru/suggests/educational_institutions?text=%D0%9C%D0%A4%D0%A2%D0%98), как это бывает.
2. Данные HH видимо не всегда поспевают за переименованиями университетов, и проходилось отлавливать университеты по старым названиям. Из того с чем столкнулся, при поиске ID-шников: «Уральский государственный медицинский университет» — «Уральская государственная медицинская академия»; «Смоленский государственный медицинский университет» — «Смоленская государственная медицинская академия»; «Тихоокеанский государственный медицинский университет» — «Владивостокский государственный медицинский университет».
3. Университеты объединяются, в этом случае в API psa.ru используется ID одной из объединённых организаций, если в данных HH нет ещё нового университета. Конкретный пример — для «Уфимский университет науки и технологий» используется «Башкирского государственного университета».
4. По каким-то причинам мне не удалось найти в HH «Университет Иннополис».

## Исходные данные об университетах

Сформируем таблицу университетов с основными данными и ID для работы с HH API.

In [19]:
# Функция для работы с API https://app.psal.ru/api
def get_results_from_api(url, page=1):
    results = pd.DataFrame()

    resp = requests.get(url + f"?page={page}")

    if resp.status_code == 200:
        json_data = resp.json()
        results = pd.concat([results, pd.json_normalize(json_data['results'])], ignore_index=True)

        if json_data['next']:
            next_page_df = get_results_from_api(url, page + 1)
            results = pd.concat([results, next_page_df], ignore_index=True)
    else:
        return pd.DataFrame()

    return results

In [39]:
# Формируем таблицу со списком университетов
universities = get_results_from_api('https://app.psal.ru/api/university/')
universities = universities[universities['deleted_at'].isnull()]
universities = universities.apply(lambda x: pd.concat([
    x[list(filter(lambda y: y in ['id', 'title_short', 'title_display', 'domain', 'educational_institution_id'], x.keys()))],
    pd.Series({'educational_institution_id': x['hh'][0]['educational_institution_id']})
]), axis=1)
universities = universities[~universities['educational_institution_id'].isnull()]
universities['educational_institution_id'] = universities['educational_institution_id'].astype(int)
universities.head(5)

,id,title_short,title_display,domain,educational_institution_id
0,1,Адыгейский государственный университет,АГУ,adygnet.ru,41900
1,2,Алтайский государственный университет,АлтГУ,asu.ru,41808
2,3,Амурский государственный университет,АмГУ,amursu.ru,42650
3,4,Астраханский государственный университет,АГУ им. В.Н. Татищева,asu.edu.ru,40229
5,6,Белгородский государственный технологический у...,БГТУ,bstu.ru,45479
